In [1]:
import os
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import re

In [2]:
dialog = pd.read_csv('DATA/dialogs_2.csv')

In [3]:
dialog

,appeal_id,datetime,client_id,user_id,msg_type,channel_id,channel_name,channel_type,answer_time,text
0,323874,2088-08-08 16:44:02+03,NaN,19670.0,0,73,Webdealer,Веб-чат,12.0,Привет.))
1,323870,2088-08-08 16:44:01+03,NaN,19671.0,0,73,Webdealer,Веб-чат,NaN,Все подписки отключены. Как это делать самосто...
2,323861,2088-08-08 16:44:01+03,NaN,19418.0,0,133,Нижний Новгород B2C,Веб-чат,NaN,Вы забыли пароль и хотите войти в кабинет?
3,323850,2088-08-08 16:44:01+03,NaN,19578.0,0,2,Viber,Viber,54.0,Можем отправить заявку на подключение новых но...
4,322276,2088-08-08 16:44:01+03,NaN,NaN,2,0,NaN,NaN,NaN,Обращение закрыто в связи с неактивностью поль...
5,323867,2088-08-08 16:43:58+03,569008.0,NaN,0,177,Омск B2C,Веб-чат,NaN,Тогда проще перейти с номером в мегафон Включа...
6,323868,2088-08-08 16:43:57+03,NaN,19671.0,0,195,Бурятия B2C,Веб-чат,18.0,По какой причине решили перейти?
7,323873,2088-08-08 16:43:56+03,NaN,19389.0,0,77,Краснодар B2C,Веб-чат,13.0,"Здравствуйте, меня зовут Оксана. С удовольстви..."
8,323876,2088-08-08 16:43:54+03,NaN,19752.0,12,79,Волгоград B2C,Веб-чат,NaN,Приветствуем Вас в службе поддержки клиентов! ...
9,323876,2088-08-08 16:43:54+03,NaN,NaN,4,0,NaN,NaN,NaN,NaN


In [172]:
dialog.columns

Index(['ID', 'Дата.изменения.статуса', 'Статус', 'Автор', 'Пользователь',
       'Категория.автора', 'Контент', 'Тип.обращения', 'Хост', 'Подписчики',
       'Тема', 'Локация', 'Тональность', 'Тип'],
      dtype='object')

In [189]:
dialog[dialog['ID'] == 282894.0]['Контент'].values

array(["'Добрый вечер! Интересует сотрудничество с вашим брендом - мы занимаемся продвижением артистов в вк, и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков (будет размещено в более 1 500 различных сообществ). Проведем фотосессии вашего смартфона с моделями, фото будут наподобие что прикрепил - с логотипом Tele2 в кадре, к фото будем добавлять подборку песен и делать посев по сообществам вк. Интересно ли вам сотрудничество такого плана?",
       nan,
       "'Олег, здравствуйте. Сообщите вашу контактную информацию. Мы передадим ее коллегам. Если их заинтересует ваше предложение, с вами обязательно свяжутся.",
       nan, nan,
       "'Контактное лицо: Степан<br>Почта xxx@mail.ru<br>Телефон +xxx xxx xx xx<br>Можно охватить аудиторию и более 90 000 000 подписчиков, но займет в два раза больше времени.<br>Также есть возможность частично бартером это организовать. Еще один вариант интеграции - мем, в таком случае можно параллельно их исп

In [196]:
dialog

,ID,Дата.изменения.статуса,Статус,Автор,Пользователь,Категория.автора,Контент,Тип.обращения,Хост,Подписчики,Тема,Локация,Тональность,Тип
0,282893.0,2088-04-02 03:01:12,Новый,token_102441,token_111457,Обычный,'Если я перейду с тарифа мой разговор на мой о...,Комментарий,'ВКонтакте,260.0,Тарифы,Москва,NaN,0-пост
1,282893.0,2088-04-02 03:46:55,Инцидент,token_NA,token_111457,NaN,NaN,NaN,NaN,NaN,Тарифы,Москва,NaN,2-инцидент
2,282893.0,2088-04-02 05:52:52,Ожидание ответа автора,token_111418,token_111438,NaN,"'[xxxxx], здравствуйте. При смене тарифного пл...",NaN,NaN,NaN,NaN,NaN,NaN,1-ответ
3,282893.0,2088-04-02 05:52:53,Ожидание ответа автора,token_NA,token_111438,NaN,NaN,NaN,NaN,NaN,Тарифы,Санкт-Петербург,NaN,2-инцидент
4,282893.0,2088-04-03 05:58:11,Закрытые,token_NA,token_111424,NaN,NaN,NaN,NaN,NaN,Тарифы,Санкт-Петербург,NaN,2-инцидент
5,282894.0,2088-04-02 03:02:50,Новый,token_89751,token_111457,Обычный,'Добрый вечер! Интересует сотрудничество с ваш...,Личное сообщение,'ВКонтакте,1.0,Прочее,NaN,NaN,0-пост
6,282894.0,2088-04-02 03:49:48,Инцидент,token_NA,token_111457,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент
7,282894.0,2088-04-02 06:16:07,Ожидание ответа автора,token_111417,token_111456,NaN,"'Олег, здравствуйте. Сообщите вашу контактную ...",NaN,NaN,NaN,NaN,NaN,NaN,1-ответ
8,282894.0,2088-04-02 06:16:07,Ожидание ответа автора,token_NA,token_111456,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент
9,282894.0,2088-04-02 11:22:58,Инцидент,token_NA,token_111424,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент


In [213]:
d = dialog[(dialog['Статус'] == 'Инцидент') | (dialog['Статус'] == 'Новый')]

In [211]:
dialog[(dialog['Статус'] == 'Инцидент') | (dialog['Статус'] == 'Новый')]

,ID,Дата.изменения.статуса,Статус,Автор,Пользователь,Категория.автора,Контент,Тип.обращения,Хост,Подписчики,Тема,Локация,Тональность,Тип
0,282893.0,2088-04-02 03:01:12,Новый,token_102441,token_111457,Обычный,'Если я перейду с тарифа мой разговор на мой о...,Комментарий,'ВКонтакте,260.0,Тарифы,Москва,NaN,0-пост
1,282893.0,2088-04-02 03:46:55,Инцидент,token_NA,token_111457,NaN,NaN,NaN,NaN,NaN,Тарифы,Москва,NaN,2-инцидент
5,282894.0,2088-04-02 03:02:50,Новый,token_89751,token_111457,Обычный,'Добрый вечер! Интересует сотрудничество с ваш...,Личное сообщение,'ВКонтакте,1.0,Прочее,NaN,NaN,0-пост
6,282894.0,2088-04-02 03:49:48,Инцидент,token_NA,token_111457,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент
9,282894.0,2088-04-02 11:22:58,Инцидент,token_NA,token_111424,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент
10,282894.0,2088-04-02 11:22:58,Инцидент,token_89751,token_NA,Обычный,'Контактное лицо: Степан<br>Почта xxx@mail.ru<...,Личное сообщение,'ВКонтакте,1.0,Прочее,NaN,NaN,0-пост
13,282894.0,2088-04-02 12:02:56,Инцидент,token_NA,token_111424,NaN,NaN,NaN,NaN,NaN,Прочее,Не определена,NaN,2-инцидент
14,282894.0,2088-04-02 12:02:56,Инцидент,token_89751,token_NA,Обычный,"'Спасибо больше! Нет, вопросов больше нет :)",Личное сообщение,'ВКонтакте,1.0,Прочее,NaN,NaN,0-пост
17,282896.0,2088-04-02 03:13:16,Новый,token_66467,token_111457,Обычный,'Теле 2 зажигай давай...,Комментарий,'ВКонтакте,17.0,Официальные блоги,Москва,NaN,0-пост
18,282896.0,2088-04-02 03:13:16,Новый,token_66467,token_111457,Обычный,'Интернет по дешевле а денег по больше из за р...,Комментарий,'ВКонтакте,17.0,Официальные блоги,Москва,NaN,0-пост


In [214]:
d = d['Контент'].dropna()

In [216]:
d.apply(lambda x: re.sub("<.*?>", " ", x)).to_csv('all_users.tsv', sep='\t', index=None)

In [227]:
d = d.apply(lambda x: re.sub("<.*?>", " ", x))

In [6]:
dialog['Тема'].value_counts()

Прочее                         756189
Официальные блоги               18380
Тарифы                          13150
Интернет                         9524
Телефония                        6154
О компании/Условия договора      4011
Базовые услуги                   3654
Роуминг                          2435
Зона покрытия                    2288
Контент                          2141
Перенос номера                   1560
Оборудование                     1169
Пакеты услуг                     1158
Обслуживание                      690
SMS                               588
Как подключиться к Tele2          444
Cпособы оплаты                    317
PIN, PUK коды                     119
Name: Тема, dtype: int64

In [15]:
dialog[dialog['Тема'] == 'Интернет']['Контент'].values

array(["'Если я перейду с тарифа мой разговор на мой онлайн когда мне нужно будет оплатить абон. Плату. Подключение у меня 25го числа, хочу перейти щас",
       nan, nan, ...,
       "'У всех бабушек есть секретный рецепт, как заманить внуков в гости. Татарская ба готовит эчпочмак, бабулxxxxxxxxxxx2 не меняется. Не то что ваш вес после бабушкиных пирожков <U+0001F605><br>Узнайте больше: vk.cc/89nY2p",
       "'Сотовый оператор Tele2 объявил об изменении оказания услуг связи на всех тарифах<br>Ситуация на российском рынке постоянно меняются. Цены на все постепенно растут, интересы и потребности людей постоянно меняются, а новые технологии, для работы с которыми требуется дорогостоящее новейшее оборудование, подливают масла в огонь. Сегодня, 22 июня",
       "'Привет. Кому уже пришла смска что с 1 июля увеличивается цена мобильной связи? Маме пришла теле 2 с 20 копеек дорожает до 1.20 за минуту. У нее старый тариф. Мне пока ничего. Анон)"],
      dtype=object)

In [233]:
def preprocess(text):
#     q = re.search('Задайте ваш вопрос или опишите проблему: (.*)', text)
#     if not q: return ''
    
#     text = q.group(1)
    tokens = [word.lower() for word in word_tokenize(text) if word.isalnum() and len(word) > 1 and len(word) < 30]
    return ' '.join(tokens)

In [148]:
def preprocess_raw(text):
    q = re.search('Задайте ваш вопрос или опишите проблему: (.*)', text)
    if not q: return ''
    
    text = q.group(1)
    return text

In [132]:
dialog

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT
0,token_7747,xxxx@yandex.ru,token_48,spb.vopros@tele2.ru,Обратная связь,2088-06-17 02:31:32,2088-06-18 19:48:26,2088-06-18 20:24:59,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\n\r..."
1,token_14226,xxxx@mail.ru,token_50,Saransk.Vopros@tele2.ru,Re[2]: Обратная связь,2088-06-17 02:31:52,2088-06-18 18:43:45,2088-06-18 18:58:43,\r\nвся необходимая информация отображена в ...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
2,token_23942,xxxx@mail.ru,token_3,rostov.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:14,2088-06-18 18:43:45,2088-06-18 18:47:06,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл..."
3,token_18368,xxxx@mail.ru,token_53,krasnodar.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:36,2088-06-18 19:49:10,2088-06-18 19:53:57,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx! ¦\r\n..."
4,token_21039,xxxx@mail.ru,token_34,kostroma.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:56,2088-06-18 19:49:17,2088-06-18 19:59:48,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nАлександр, доброго времени суток! ;)\r..."
5,token_24778,xxxx@mal.ru,token_103,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:37:23,2088-06-18 19:52:46,2088-06-18 20:03:32,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nСп..."
6,token_12422,xxxx@gmail.com,token_69,nnov.vopros@tele2.ru,Обратная связь,2088-06-17 02:37:36,2088-06-18 20:53:39,2088-06-18 20:57:10,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
7,token_24424,xxxx@mail.ru,token_40,vladimir.vopros@tele2.ru,"Заполнена форма ""Ск<U+FFFD><U+FFFD>рая абонент...",2088-06-17 02:37:56,2088-06-18 20:55:16,2088-06-18 20:59:46,"Заполнена форма ""Скорая абонентская""\r\n\r\nms...","\r\n\r\nПриветствую Вас, xxxxx xxxxx xxxxx!\r\..."
8,token_19238,xxxx@gmail.com,token_28,Perm.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:02,2088-06-18 18:44:19,2088-06-18 18:51:39,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"Здравствуйте, Василий Владимирович!\r\nСпасибо..."
9,token_19252,xxxx@mail.ru,token_69,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:18,2088-06-18 20:57:11,2088-06-18 21:10:10,Тело письма\r\n\r\nMSISDN: xxx xxx xx xx\r\nО...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."


In [161]:
texts = dialog['CONTENT'].dropna().iloc[:50000].apply(preprocess)

In [234]:
texts = d.apply(preprocess).values

In [162]:
texts_raw = dialog['CONTENT'].dropna().iloc[:50000].apply(preprocess_raw).values

In [222]:
tfidf = TfidfVectorizer(max_features=25000,min_df=100, max_df=0.3, analyzer='char', ngram_range=(2,5))

In [235]:
texts

array(['перейду тарифа мой разговор на мой онлайн когда мне нужно будет оплатить абон плату подключение меня 25го числа хочу перейти щас',
       'вечер интересует сотрудничество вашим брендом мы занимаемся продвижением артистов вк можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено более 500 различных сообществ проведем фотосессии вашего смартфона моделями фото будут наподобие что прикрепил логотипом tele2 кадре фото будем добавлять подборку песен делать посев по сообществам вк интересно ли вам сотрудничество такого плана',
       'лицо степан почта xxx телефон xxx xx xx можно охватить аудиторию более 90 000 000 подписчиков но займет два раза больше времени также есть возможность частично бартером это организовать еще один вариант интеграции мем таком случае можно параллельно их использовать там разные аудитории общей сумме 90 000 000 подписчиков займет такое же время',
       ..., 'где связь алею', 'ее нет', 'лесу ловит г

In [223]:
vecs = tfidf.fit_transform(d.values)

In [ ]:
kf = KFold(n_splits=20)


In [155]:
ft = gensim.models.FastText.load('fast_text')

In [236]:
vecs_ft = np.zeros((len(texts), 200))

for i, sent in enumerate(texts):
    words = sent.split()
    v = np.zeros((len(words), 200))
    weights = []
    for j, word in enumerate(words):
        try:
            v[j] = ft[word]
            weights.append(1)
        except KeyError:
            weights.append(0)
            continue
    if not v.any():
        continue
    v = np.average(v, axis=0, weights=weights)
    vecs_ft[i] = v

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [70]:
vecs_ft[np.isnan(vecs_ft)]

array([], dtype=float64)

In [237]:
km = MiniBatchKMeans(5000, verbose=1)
km.fit(vecs_ft)

Init 1/3 with method: k-means++


/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)


Inertia for init 1/3: 1585.821586
Init 2/3 with method: k-means++
Inertia for init 2/3: 1028.941331
Init 3/3 with method: k-means++
Inertia for init 3/3: 1060.987374
Minibatch iteration 1/187800: mean batch inertia: 46.231144, ewa inertia: 46.231144 
Minibatch iteration 2/187800: mean batch inertia: 53.664915, ewa inertia: 46.239063 
Minibatch iteration 3/187800: mean batch inertia: 44.755648, ewa inertia: 46.237483 
Minibatch iteration 4/187800: mean batch inertia: 54.496471, ewa inertia: 46.246281 
Minibatch iteration 5/187800: mean batch inertia: 51.888999, ewa inertia: 46.252292 
Minibatch iteration 6/187800: mean batch inertia: 54.169406, ewa inertia: 46.260726 
Minibatch iteration 7/187800: mean batch inertia: 56.210819, ewa inertia: 46.271325 
Minibatch iteration 8/187800: mean batch inertia: 45.244889, ewa inertia: 46.270232 
Minibatch iteration 9/187800: mean batch inertia: 60.198887, ewa inertia: 46.285070 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=5000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=1)

In [170]:
dialog

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT
0,token_7747,xxxx@yandex.ru,token_48,spb.vopros@tele2.ru,Обратная связь,2088-06-17 02:31:32,2088-06-18 19:48:26,2088-06-18 20:24:59,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\n\r..."
1,token_14226,xxxx@mail.ru,token_50,Saransk.Vopros@tele2.ru,Re[2]: Обратная связь,2088-06-17 02:31:52,2088-06-18 18:43:45,2088-06-18 18:58:43,\r\nвся необходимая информация отображена в ...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
2,token_23942,xxxx@mail.ru,token_3,rostov.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:14,2088-06-18 18:43:45,2088-06-18 18:47:06,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл..."
3,token_18368,xxxx@mail.ru,token_53,krasnodar.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:36,2088-06-18 19:49:10,2088-06-18 19:53:57,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx! ¦\r\n..."
4,token_21039,xxxx@mail.ru,token_34,kostroma.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:56,2088-06-18 19:49:17,2088-06-18 19:59:48,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nАлександр, доброго времени суток! ;)\r..."
5,token_24778,xxxx@mal.ru,token_103,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:37:23,2088-06-18 19:52:46,2088-06-18 20:03:32,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nСп..."
6,token_12422,xxxx@gmail.com,token_69,nnov.vopros@tele2.ru,Обратная связь,2088-06-17 02:37:36,2088-06-18 20:53:39,2088-06-18 20:57:10,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
7,token_24424,xxxx@mail.ru,token_40,vladimir.vopros@tele2.ru,"Заполнена форма ""Ск<U+FFFD><U+FFFD>рая абонент...",2088-06-17 02:37:56,2088-06-18 20:55:16,2088-06-18 20:59:46,"Заполнена форма ""Скорая абонентская""\r\n\r\nms...","\r\n\r\nПриветствую Вас, xxxxx xxxxx xxxxx!\r\..."
8,token_19238,xxxx@gmail.com,token_28,Perm.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:02,2088-06-18 18:44:19,2088-06-18 18:51:39,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"Здравствуйте, Василий Владимирович!\r\nСпасибо..."
9,token_19252,xxxx@mail.ru,token_69,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:18,2088-06-18 20:57:11,2088-06-18 21:10:10,Тело письма\r\n\r\nMSISDN: xxx xxx xx xx\r\nО...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."


In [238]:
labels = km.labels_
cls = defaultdict(list)
texts_raw = d.values
for i, l in enumerate(labels):
    cls[l].append(texts_raw[i])

In [116]:
vecs.shape

(10000, 5132)

In [232]:
f = open('кластеры/вопросы.txt', 'w')
for cl in cls:
    f.write('######Кластер --  ' + str(cl) + '\n')
    f.write('\n'.join([x for x in cls[cl]]))
    f.write('\n\n\n')
f.close()

In [154]:
dialog

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT
0,token_7747,xxxx@yandex.ru,token_48,spb.vopros@tele2.ru,Обратная связь,2088-06-17 02:31:32,2088-06-18 19:48:26,2088-06-18 20:24:59,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\n\r..."
1,token_14226,xxxx@mail.ru,token_50,Saransk.Vopros@tele2.ru,Re[2]: Обратная связь,2088-06-17 02:31:52,2088-06-18 18:43:45,2088-06-18 18:58:43,\r\nвся необходимая информация отображена в ...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
2,token_23942,xxxx@mail.ru,token_3,rostov.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:14,2088-06-18 18:43:45,2088-06-18 18:47:06,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл..."
3,token_18368,xxxx@mail.ru,token_53,krasnodar.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:36,2088-06-18 19:49:10,2088-06-18 19:53:57,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx! ¦\r\n..."
4,token_21039,xxxx@mail.ru,token_34,kostroma.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:56,2088-06-18 19:49:17,2088-06-18 19:59:48,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nАлександр, доброго времени суток! ;)\r..."
5,token_24778,xxxx@mal.ru,token_103,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:37:23,2088-06-18 19:52:46,2088-06-18 20:03:32,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nСп..."
6,token_12422,xxxx@gmail.com,token_69,nnov.vopros@tele2.ru,Обратная связь,2088-06-17 02:37:36,2088-06-18 20:53:39,2088-06-18 20:57:10,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
7,token_24424,xxxx@mail.ru,token_40,vladimir.vopros@tele2.ru,"Заполнена форма ""Ск<U+FFFD><U+FFFD>рая абонент...",2088-06-17 02:37:56,2088-06-18 20:55:16,2088-06-18 20:59:46,"Заполнена форма ""Скорая абонентская""\r\n\r\nms...","\r\n\r\nПриветствую Вас, xxxxx xxxxx xxxxx!\r\..."
8,token_19238,xxxx@gmail.com,token_28,Perm.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:02,2088-06-18 18:44:19,2088-06-18 18:51:39,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"Здравствуйте, Василий Владимирович!\r\nСпасибо..."
9,token_19252,xxxx@mail.ru,token_69,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:18,2088-06-18 20:57:11,2088-06-18 21:10:10,Тело письма\r\n\r\nMSISDN: xxx xxx xx xx\r\nО...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
